In [1]:
import bz2
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as snb

**Exercise 1**

Count the number of lines in Python for
each file

In [2]:
! ls cd ../

ls: cd: No such file or directory
../:
bookings.csv.bz2 ds_challenge     searches.csv.bz2


In [3]:
#f_in = open("../bookings.csv.bz2", "rb")
#f_out = open("bookings.csv", "wb")
#f_out.write(bz2.decompress(f_in.read()))
#f_out.close()
#f_in.close()

In [4]:
bz_file = bz2.BZ2File("../bookings.csv.bz2")
line_count = bz_file.readlines()
len(line_count)

KeyboardInterrupt: 

In [41]:
#! wc -l bookings.csv

In [39]:
bz_file = bz2.BZ2File("../searches.csv.bz2")
line_count_searches = bz_file.readlines()
len(line_count_searches)

20390199

**Exercise 2**

Top 10 arrival airports in the world in 2013 (using the bookings file)
* Arrival airport is the column arr_port. It is the IATA code for the airport
* To get the total number of passengers for an airport, you can sum the column
pax, grouping by arr_port. Note that there is negative pax. That corresponds to
cancelations. So to get the total number of passengers that have actually
booked, you should sum including the negatives (that will remove the canceled
bookings).
* Print the top 10 arrival airports in the standard output, including the number of
passengers.
* Bonus point: Get the name of the city or airport corresponding to that airport
(programatically, we suggest to have a look at GeoBases in Github)
* Bonus point: Solve this problem using pandas (instead of any other approach)

In [5]:
columns_arr_ranking = ['arr_port','year','pax']

In [6]:
pd.options.display.max_columns = None
bookings = pd.read_csv('bookings.csv', nrows=100000, sep='^', na_values='NULL', usecols=columns_arr_ranking)
bookings.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


In [7]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   arr_port  100000 non-null  object
 1   pax       100000 non-null  int64 
 2   year      100000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [8]:
arr_pax = bookings[bookings['year'] == 2013].groupby(by='arr_port').sum('pax').sort_values('pax', ascending=False)

In [15]:
top_10_arr = arr_pax.drop('year', axis=1).head(10)
top_10_arr

,pax
arr_port,
LHR,1006
MCO,861
JFK,795
LAX,761
BKK,747
LAS,732
SFO,705
ORD,686
CDG,676


In [10]:
'''data_iterator = pd.read_csv('bookings.csv', nrows=100000, sep='^', na_values='NULL', usecols=columns_arr_ranking, chunksize=10**6)

chunk_list = []  

# Each chunk is in dataframe format
for data_chunk in data_iterator:  
    filtered_chunk = chunk_filtering(data_chunk) # own function
    chunk_list.append(filtered_chunk)

filtered_data = pd.concat(chunk_list)'''

"data_iterator = pd.read_csv('bookings.csv', nrows=100000, sep='^', na_values='NULL', usecols=columns_arr_ranking, chunksize=10**6)\n\nchunk_list = []  \n\n# Each chunk is in dataframe format\nfor data_chunk in data_iterator:  \n    filtered_chunk = chunk_filtering(data_chunk) # own function\n    chunk_list.append(filtered_chunk)\n\nfiltered_data = pd.concat(chunk_list)"

In [19]:
data_iterator = pd.read_csv('bookings.csv', sep='^', na_values='NULL', usecols=columns_arr_ranking, chunksize=10**6)

chunk_list = []  

# Each chunk is in dataframe format
for data_chunk in data_iterator:  
    #filtered_chunk = chunk_filtering(data_chunk) # own function
    chunk_list.append(data_chunk)

filtered_data = pd.concat(chunk_list)

In [20]:
arr_pax = filtered_data[filtered_data['year'] == 2013].groupby(by='arr_port').sum('pax').sort_values('pax', ascending=False)

In [21]:
arr_pax

,pax,year
arr_port,,
LHR,88809.0,433904163.0
MCO,70930.0,180586230.0
LAX,70530.0,288301860.0
LAS,69630.0,187933680.0
JFK,66270.0,319926090.0
...,...,...
MYJ,-80.0,161040.0
UPG,-90.0,905850.0
XWG,-90.0,261690.0


In [22]:
top_10_arr = arr_pax.drop('year', axis=1).head(10)
top_10_arr

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


In [23]:
top_10_arr.to_csv("top_10_arr.csv")